# Implémenter une API dans Flask

## Introduction

Les applications qui triatent des données sont rarement (ou très partiellement) des applications web traditionnelles. Très souvent :
- soit elles fourniront des données via une API
- soit elles s'inscriront dans un processus, comme composnt d'une application plus vaste
- soit elles seront reliées à d'autres applications par des systèmes de mesageries

Dans ce module, nous allons examiner le prmier cas de figure, et même en le restreignant aux API REST, qui ne sont pas les seules sur les réseaux.

Depuis que REST a été défini, comme modèle a posteriori d'Internet, des travaux ont été menés pour donner une description formelle d'une API. Cela a a bouti à une application du nom de **Swagger**, qui a ensuite été adotée comme un standard par la communauté sous le nom d'__Open API__.

Swagger est un outil intéressant, car il établit une spécification normalisée des API REST, et fournit des outils d'édition et de test (ce qui est plus global qu'un autre outil comme Postman, par exemple, nonobstant ses qualités).

## Mise en œuvre

### Installation

Pour pouvoir utiliser Swagger dans notre application Flask, nosu allons devoir installer le module _ad hoc_, développé et mis à disposition par le groupe Zalando.
```bash
pip install connexion[swagger-ui]
```
`connexion` est le nom du module, et oninf-dique entre crochet les options à ajouter, ici `swagger-ui` qui permettra de tester l'API directement depuis l'application.

### Conception de l'API

Pour tout ce qui est de la spécification Open API, nous vous renvoyons au document qui en parle. Pour l'éditer, vous avez le choix entre l'[éditeur en ligne de Swagger](https://editor.swagger.io), soit la version de bureau que vous pouvez télécharger.

Une fois votre API spécifiée, vous pouvez l'exporter dans un fichier YAML que vous déposez à la racine de votre application.

### Configuration de l'application

TODO

## Swagger / Open API


### Structure de base

La structure d'un spécification d'API selon Open API se découpe en plusieurs sections que nous allons voir en détail.

#### Métadonnées

La première section regroupe les informations « documentaires » sur l'API :
```yaml
# version d'Open API
openapi: 3.0.0

info:
    title: API pour ClassicModels
    description: L’API met à disposition d'application tierces les données de ClassicModels
    version: 0.5.2
    contact:
        email: 'michel.cadennes@sens-commun.paris'
    license:
        name: 'GPL 3.0'

```

#### Serveurs

Les serveurs sont les points d'accès de l'API. Vous pouvez en indiquer autant que vous voulez, par exemple pour des usages différents (production, démo, bac à sable, etc.)
```yaml
servers:
    - url: "https://api.classicmodels.com
      description: Serveur de référence
```

#### Composants

La section `components` recense des éléments réutilisables de l'API. Par « réutilisable », on entend que les objets définis dans cette sections peuvent importés par leur nom dans d'autres partiesde la spécification.

> [La section `components`](https://swagger.io/docs/specification/components/)

Comme on peut le voir dans la documentation, ceci recouvre toute une série d'objets différents. En particulier, il existe une sous-section `schemas`, qui décrit les entités du modèle du domaine de l'application. La description de ces entités ne difère pas beaicoup des habitudes des langages objets ou SQL. Elle recense la liste des propriétés de l’entité et leurs attributs. Par exemple: 
```yaml
components:
    schemas:
        Office:
            type: object
            properties:
                officeNumber:
                    type: integer
                city:
                    type: string
                    example: 'San Francisco'
            required:
                - id
                - city
```
Dans l’exemple ci-dessus,vous définissons une entité `Office`, qui a deux propriétés, `id` qui est un entier, et `city` qui est une chaîne de caractères. Les deux sont obligatoires (non « _nullable_ »), et nous donnons un exemple d'utilisation.

Less types des propriétés sont très variées, comme indiqué dans [page de la documentation](https://swagger.io/docs/specification/data-models/data-types/) et comprennent des scalaires (`integer`, `string`, etc.) mais aussi des types composés, comme des listes ou des objets.

Chaque type admet des attributs complémentaires. Par exemple :
- les nombres (`number`) peuvent avoir une valeur minimale et/ou une valeur maximale
- les chaines de caractères peuvent être décrites par une expression régulière

Par ailleurs, il est possible de définir des types « algébriques », comme somme de types :
```yaml
# ...
isAvailable:
    oneOf:
      - integer
      - boolean
```
Dans cet exemple, nous spécifions que la propriété « est disponible » peut être représentée soit par `True`/`False`, soit par `0`/`x `.

##### Associations

DAns les modèles, les entités sont très souvent liées entre elles par des asociations. Or, dans Open API, le type d'une propriété ne peut être que `object` dans un sens très générique. Nous avons alors besoin d'une _référence_ à un type d'objet précis. Pour ce faire, OpenAPI introduit le marqueur `ref`, que l'on retrouve dans divers usages, mais qui désigne dans les modèles les associations. Par exemple, la retation entre `Office` et `Employee` :

```yaml
components:
    schemas:
        Office:
            type: object
            # ... suite de la définition
        Employee:
            type: object       
            properties:
                id:
                    type: integer
                officeNumber:
                    ref : '#/components/schemas/Office'

```
La valeur de la référence s'interprète facilement comment une arborescence qui suit la structure de `components`, le `#` signifiant la racine du document. Il est possible de faire référence à des documents externes, voire à des URL :
```yaml
# fichier local
ref: '../../office.yaml#/components/schemas/Office'
# serveur distant
ref: 'http://domain.com/files/office.yaml#/components/schemas/Office'
```

> [Commet utiliser $ref](https://swagger.io/docs/specification/using-ref/)

##### Autres définitions

Parmi les objets réutilisables définis dans la section `components`, nous trouvons aussi des informations sont il sera question juste après dans la défintion des routes :
- les paramètres
- les corps de requêtes
- les réponses


#### Routes

La partie principale de la spécification est naturellement l'ensemble des routes accessible à des requêtes HTTP. Elles sont rassemblées sous la section `paths`.

La description d'une route comprend :
- un schéma d'URL, pouvant contenir des variables
- les méthodes HTTP compatibles
- les paramètres de la route, correspondant aux variables de l'URL
- la procédure (ou opération) déclenchée par la requête
- le corps de la requête
- la forme de la réponse

Ce à quoi il est recommandé d'associer :
- une description courte de la fonction de la route
- une description longue, au format Markdown (ou HTML) donc éventuellement avec des liens
- un lien vers une documentation externe
- éventuellement des « _tags_ », qui sont des méta-données partagées décrites dans une section spéciale

> **N.B.** Pour les bonnes pratiques sur la conception d'API, se reporter au mémento d'OCTO inclus dans les ressources documentaires

Admettons que nous vouilons définir un fragment d'API destiné à gérer les données d'un employé particulier. Nous allons définir un schéma d'URL qui pourrait être : `/employee/{id}`.

Ce schéma mutualisera les différentes opérations réalisables, typiquement celles du CRUD : `GET`, `PATCH`, `DELETE`. , `POST` étant un cas à part, son URL ne peut évidemment pas comporter d'identifiant.

Le squelette de notre spécification ressemblera à :
```yaml
# La section des routes
paths:
    # Le schéma d'URL de la route à définir
    /employee/{id}:
        # Les méthodes utilisables
        get:
            # spécification GET
        patch:
            # spécification PATCH
        delete:
            # spécification DELETE
```
Pour chaque méthode, nous devons maintenant définir les constituants de la route. Pour `get`, par exemple :
```yaml
paths:
    /employee/{id}:
        get:
            # Description courte
            summary: Profil d’un employé
            # Description longue
            description: Les employés du magasin ont un bureau et prennent en charge les clients
            # Le paramètre de la route
            parameters:
                - name: id
                  # Le paramètre est intégré dans l'URL (et non dans la chaîne de requête)
                  in: path
                  required: true
                  description: L’identifiant de l’employé
                  # Type du paramètre, analogue aux schémas des composants
                  schema:
                      type: integer
                      format: int64
                      minimum: 1
            responses:
                '200':
                    description: 'OK'
                    content:
                        # Les différents formats (Content-Type) de la réponse
                        # Ici on ne décrit que JSON, mais le nombre autorisé est arbitraire
                        application/json:
                            # Descriptindu contenu de la réponse
                            schema:
                                type: object
                                properties:
                                    id:
                                        type: integer
                                        format: int64
                                        example: 4
                                    name:
                                        type: string
                                        example: Jessica Smith
                '400':
                    description: 'L’identifiant n’est pas au bon format'
                '404':
                    desscription: 'Employé inconnu'
```
Comme on le voit, une définition précise de l'API peut être très longue et verbeuse (mais, en même temps, c'est aussi ce que l'on cherche).

Dans l'exemple ci-dessus, nous voyons quenous pourions mutualiser certains éléments de la spécification. Par exemple, le paramètre (`id`) se retrouvera dans l'ensemble des méthodes et peut-être au-delà. On onverra peut-être aussi les mêmes réponses pour différents points d'entrée de l'API. Tout ce qui peut être mutualisé devrait être déporté dans la section `documents`. Ainsi pour le paramètre :

a) Description dans les composants :
```yaml
components:
# ...
    parameters:
        # Le paramètre est nommé
        EmployeeID:
            - name: id
              in: path
              required: true
              description: L’identifiant de l’employé
              schema:
                  type: integer
                  format: int64
                  minimum: 1
# ...
```
b) Utilisation dans la route :
```yaml
paths:
    /employee/{id}:
        get:
            # ...
            parameters:
                # La description du paramètre est remplécée par sa référence
                $ref: '#/components/parameters/EmployeeID'
```

Il est possible de reproduire le même mécanisme avec les réponses, typiquement l'erreur 404 qui se répétera un peu partout :

```yaml
components:
    responses:
        404NotFound:
            description: 'La ressource demandée n’a pas été trouvée'
```
Puis :
```yaml
paths:
    /employee/{id}:
        get:
            # ...
            parameters:
                # La description du paramètre est remplécée par sa référence
                $ref: '#/components/parameters/EmployeeID'
            responses:
                404:
                    $ref: '#/components/responses/404Notfound'
```

En sens inverse, si nous voulons modifierle profil d'un employé, nous allons devoir décrire une route pour la méthode `POST`, précisant quelles sont les données que nous recevrons dans un tel cas de figure. Open API offre pour cela l'élément `requestBody` :
```yaml
paths:
    /employee/{id}:
        patch:
            # ...
            parameters:
                # La description du paramètre est remplécée par sa référence
                $ref: '#/components/parameters/EmployeeID'
            # Le contenu du corps de la requête
            requestBody:
                # Plusieurs cas peuvnt être examinés
                application/json:
                    # Une requête AJAX, par exemple, attend un objet conforme à l'entité `Employee`
                    $ref: '#/components/schemas/Employee'
                application/x-www-form-urlencoded:
                    # Un formulaire autorisant la modification du téléphone et du numéro de bureau
                    schema:
                        type: object
                        properties:
                            phone:
                                type: string
                            officeNumber:
                                type: string
```
Encore une fois, les spécifications de corps de requêtes peuvent être mutualisés dans l'élément `requestBodies` de la section `components` .
